## Tensor Flow Problem Sheet

These problems relate to the [TensorFlow](https://www.tensorflow.org/) python library for pattern recognition. This notebook uses the [Iris Data Set](https://archive.ics.uci.edu/ml/datasets/iris).

### 1.  Use Tensorflow to create a model.
Create a model that uses a flower's sepal width / length and petal width / length to predict the species of Iris.

In [19]:
import tensorflow as tf
import pandas as pd
import numpy as np
from numpy import genfromtxt
iD = genfromtxt('data.csv', delimiter=',')
irisData = np.array([iD])
with tf.Session() as sess:
    tf.global_variables_initializer().run()

    irisTF = tf.constant(irisData)
    print(irisTF.eval())

# Read data from CSV adapted from https://www.tensorflow.org/versions/r1.1/programmers_guide/reading_data




[[[ 5.1  3.5  1.4  0.2]
  [ 4.9  3.   1.4  0.2]
  [ 4.7  3.2  1.3  0.2]
  [ 4.6  3.1  1.5  0.2]
  [ 5.   3.6  1.4  0.2]
  [ 5.4  3.9  1.7  0.4]
  [ 4.6  3.4  1.4  0.3]
  [ 5.   3.4  1.5  0.2]
  [ 4.4  2.9  1.4  0.2]
  [ 4.9  3.1  1.5  0.1]
  [ 5.4  3.7  1.5  0.2]
  [ 4.8  3.4  1.6  0.2]
  [ 4.8  3.   1.4  0.1]
  [ 4.3  3.   1.1  0.1]
  [ 5.8  4.   1.2  0.2]
  [ 5.7  4.4  1.5  0.4]
  [ 5.4  3.9  1.3  0.4]
  [ 5.1  3.5  1.4  0.3]
  [ 5.7  3.8  1.7  0.3]
  [ 5.1  3.8  1.5  0.3]
  [ 5.4  3.4  1.7  0.2]
  [ 5.1  3.7  1.5  0.4]
  [ 4.6  3.6  1.   0.2]
  [ 5.1  3.3  1.7  0.5]
  [ 4.8  3.4  1.9  0.2]
  [ 5.   3.   1.6  0.2]
  [ 5.   3.4  1.6  0.4]
  [ 5.2  3.5  1.5  0.2]
  [ 5.2  3.4  1.4  0.2]
  [ 4.7  3.2  1.6  0.2]
  [ 4.8  3.1  1.6  0.2]
  [ 5.4  3.4  1.5  0.4]
  [ 5.2  4.1  1.5  0.1]
  [ 5.5  4.2  1.4  0.2]
  [ 4.9  3.1  1.5  0.1]
  [ 5.   3.2  1.2  0.2]
  [ 5.5  3.5  1.3  0.2]
  [ 4.9  3.1  1.5  0.1]
  [ 4.4  3.   1.3  0.2]
  [ 5.1  3.4  1.5  0.2]
  [ 5.   3.5  1.3  0.3]
  [ 4.5  2.3  1.

### 2. Split the data into training and testing sets.
Investigate the *best way to do this* - write some code to randomly separate data if desired. Reference relevant material.

### 3. Train the model.
Train the model using the test (should be training?) set.

### 4. Test the model.
Test your model using the testing set. Calculate and display clearly the error rate.